In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as plt
import seaborn as sb
from pylab import rcParams
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn import datasets
from influxdb import InfluxDBClient

In [2]:
df1=pd.read_csv("C:/Users/Swarupa/Desktop/Terrorism.csv")

C:\Users\Swarupa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=df1[['eventid','iyear','ransompaid','ransompaidus','nkill','nkillus','country_txt']]

In [4]:
df.dtypes

eventid           int64
iyear             int64
ransompaid      float64
ransompaidus    float64
nkill           float64
nkillus         float64
country_txt      object
dtype: object

In [5]:
df.head()

,eventid,iyear,ransompaid,ransompaidus,nkill,nkillus,country_txt
0,197000000001,1970,NaN,NaN,1.0,NaN,Dominican Republic
1,197000000002,1970,NaN,NaN,0.0,NaN,Mexico
2,197001000001,1970,NaN,NaN,1.0,NaN,Philippines
3,197001000002,1970,NaN,NaN,NaN,NaN,Greece
4,197001000003,1970,NaN,NaN,NaN,NaN,Japan


In [10]:
df.fillna(0,inplace=True)

C:\Users\Swarupa\Anaconda3\lib\site-packages\pandas\core\frame.py:2852: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [11]:
grouped=df.groupby(["iyear","country_txt"]).agg({'eventid':'count', 'ransompaid': 'sum', 'ransompaidus': 'sum', 'nkill': 'sum', 'nkillus': 'sum'}).reset_index().rename(columns={'eventid':'Number'})


In [12]:
grouped.head()

,iyear,country_txt,Number,ransompaid,ransompaidus,nkill,nkillus
0,1970,Argentina,21,0.0,0.0,5.0,0.0
1,1970,Australia,1,0.0,0.0,0.0,0.0
2,1970,Belgium,1,0.0,0.0,0.0,0.0
3,1970,Bolivia,1,0.0,0.0,0.0,0.0
4,1970,Brazil,6,0.0,0.0,2.0,0.0


In [13]:
grouped['Number']=(grouped['Number']/grouped['Number'].max())*100
grouped['nkill']=(grouped['nkill']/grouped['nkill'].max())*100
grouped['nkillus']=(grouped['nkillus']/grouped['nkillus'].max())*100
grouped['ransompaid']=(grouped['ransompaid']/grouped['ransompaid'].max())*100
grouped['ransompaidus']=(grouped['ransompaidus']/grouped['ransompaidus'].max())*100

In [14]:
grouped.head()

,iyear,country_txt,Number,ransompaid,ransompaidus,nkill,nkillus
0,1970,Argentina,0.534896,0.0,0.0,0.038229,0.0
1,1970,Australia,0.025471,0.0,0.0,0.000000,0.0
2,1970,Belgium,0.025471,0.0,0.0,0.000000,0.0
3,1970,Bolivia,0.025471,0.0,0.0,0.000000,0.0
4,1970,Brazil,0.152827,0.0,0.0,0.015292,0.0


In [15]:
grouped.to_csv("Terrorism_normalized.csv",index=False)

In [16]:
grouped.head()

,iyear,country_txt,Number,ransompaid,ransompaidus,nkill,nkillus
0,1970,Argentina,0.534896,0.0,0.0,0.038229,0.0
1,1970,Australia,0.025471,0.0,0.0,0.000000,0.0
2,1970,Belgium,0.025471,0.0,0.0,0.000000,0.0
3,1970,Bolivia,0.025471,0.0,0.0,0.000000,0.0
4,1970,Brazil,0.152827,0.0,0.0,0.015292,0.0


In [18]:
grouped['ransompaid'].max()

100.0

In [17]:
def read_data():
    with open('C:/Users/Swarupa/Desktop/Terrorism_normalized.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[1:]]
def read_header():
    with open('C:/Users/Swarupa/Desktop/Terrorism_normalized.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[:1]]

def main(host='localhost', port=8086):       #input your port number which has been set for influx
    """Instantiate the connection to the InfluxDB client."""
    user = 'Swarupa'
    password = ''
    dbname = 'SocialWeather'
    # Temporarily avoid line protocol time conversion issues #412, #426, #431.
    protocol = 'json'
    client = InfluxDBClient(host, port, user, password, dbname)
    
    print("Create DataFrame");
    b=read_header();
    length=len(b);
    print(b);
    narr=(b[-1])
    narr[-1]=narr[-1].split('\n',1)[0]
    print(narr); # getting /n (new line?,why?)
    a = read_data();
    for metric in a:
        for i in range(len(narr)):
            influx_metric = [{
                'measurement': 'Terrorism_normalized',
                'tags': {
                    'data_normalized?': 'normalized'                 },
                'time':str(metric[0])+"-12-31"+"T00:00:00Z",
                'fields': {
                    narr[i]: metric[i].split('\n',1)[0]

                }
            }]
            client.write_points(influx_metric)

    
if __name__ == '__main__':
    main()

Create DataFrame
[['iyear', 'country_txt', 'Number', 'ransompaid', 'ransompaidus', 'nkill', 'nkillus\n']]
['iyear', 'country_txt', 'Number', 'ransompaid', 'ransompaidus', 'nkill', 'nkillus']


ConnectionError: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /write?db=SocialWeather (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015DAF8D65F8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))